In [ ]:
import os
os.chdir(r'C:\Users\maint4\My Drive\MCH Sale forecasting\Works\data_SO_SI_weekly_by_channel')

In [ ]:
# pipeline
from imblearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
# data manipulation
import numpy as np
import pandas as pd
from math import sqrt
import isoweek

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# time
import datetime as dt
import time as time
import calendar
import holidays
# pipeline


# models
from sklearn.ensemble import RandomForestRegressor

# optimization and hyperparameters
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, ParameterGrid



# model evaluation

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error,mean_squared_error

from statsmodels.tsa.seasonal import seasonal_decompose

# saving models
from joblib import dump

# models
from sklearn.ensemble import RandomForestRegressor

# optimization and hyperparameters
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

# model evaluation
from sklearn.metrics import mean_squared_error


import os
import glob
file_path = r'C:\Users\maint4\My Drive\MCH Sale forecasting\Works\data_SO_SI_weekly_by_channel'
#list all the files from the directory
file_list = os.listdir(file_path)

csv_files = glob.glob('*.{}'.format('csv'))
#combine all files in the list
df_20_23 = pd.read_csv('BQ_GT_2301_BYITEMCODE.csv') 
inventory_df = pd.read_csv('BQ_GT_INVEN_2301_BYITEMCODE.csv')
#pd.concat([pd.read_csv(f) for f in file_list])
Tet_Holiday = pd.read_excel('Master_Date.xlsx', sheet_name= 'Holiday_count') #local file/can upload to gcp
Tet = pd.read_excel('Master_Date.xlsx', sheet_name= 'Tet_Holiday') #local
Vegan = pd.read_excel('Master_Date.xlsx', sheet_name= 'Vegan_Week') #local
DP_mapping = pd.read_excel('Master_Data.xlsx') #local

In [ ]:

#SO preparation
df_20_23['WEEK_NUM'] = df_20_23['WEEK_NUM_SI'].copy()
df_20_23['WEEK_NUM'] = df_20_23['WEEK_NUM'].fillna(df_20_23['WEEK_NUM_SO'])

df_20_23['ITEM_CODE'] = df_20_23['ITEM_CODE'].fillna(df_20_23['ITEM_CODE_1'])
df_20_23['YEAR'] = df_20_23['YEAR'].fillna(df_20_23['YEAR_1'])
df_20_23['CITY'] = df_20_23['CITY'].fillna(df_20_23['CITY_1'])

df_20_23['WEEKLY_SO_ORDER'] = df_20_23['WEEKLY_SO_ORDER'].fillna(0)
df_20_23['WEEKLY_SI_ORDER'] = df_20_23['WEEKLY_SI_ORDER'].fillna(0)

df = df_20_23.drop(['WEEK_NUM_SI','WEEK_NUM_SO','YEAR_1','CITY_1','ITEM_CODE_1'], axis = 1)
df.dropna(inplace = True)

In [ ]:
df['ITEM_CODE'] = df['ITEM_CODE'].str.replace('Z$','',regex=True)

In [ ]:
df_v2 = df.merge(DP_mapping[['Product Number', 'Sub Division Name', 'CATEGORY_DESC','SUB_CATEGORY_DESC',
                             'Demand Planning Standard SKU Name']], how = 'inner', 
                 left_on = 'ITEM_CODE', right_on = 'Product Number')

In [ ]:
df_v2.rename(columns = {'Demand Planning Standard SKU Name' : 'DP_STANDARD_SKU_DESC', 'Sub Division Name' : 'SUB_DIVISION_DESC'}, inplace = True)

In [ ]:
df_v2

In [ ]:
sum(df_v2[(df_v2['YEAR']==2022) & (df_v2['DP_STANDARD_SKU_DESC']=='Chilli Sauces Chinsu 1kg')]['WEEKLY_SO_ORDER'])

# Impulsing missing weeks in order of time

In [ ]:
def week_by_year(n_YEAR):
    WEEK_NUM = []
    YEAR = []
    for i in n_YEAR:
        for j in range(1,isoweek.Week.last_week_of_year(int(i)).week+1):
            YEAR.append(i)
            WEEK_NUM.append(j)
    list_of_tuples = list(zip(YEAR, WEEK_NUM))

    return pd.DataFrame(list_of_tuples,  columns= ['YEAR','WEEK_NUM'])

def make_data_continuous(df):
    STANDARD_SKU_CITY = df[['DP_STANDARD_SKU_DESC','SUB_DIVISION_DESC','CATEGORY_DESC','SUB_CATEGORY_DESC','CITY']].drop_duplicates()
    n_YEAR = df['YEAR'].unique()
    WEEK_NUM_YEAR = week_by_year(n_YEAR)
    data = pd.merge(STANDARD_SKU_CITY, WEEK_NUM_YEAR, how = 'cross')
    return pd.merge(data,df, how = 'left', on = ['DP_STANDARD_SKU_DESC', 'CITY', 'WEEK_NUM', 'YEAR','SUB_DIVISION_DESC','CATEGORY_DESC','SUB_CATEGORY_DESC'])
    
def create_period_tag(df):
    year_week_1 = df['YEAR'].astype(int).astype(str) +'-' + df['WEEK_NUM'].astype(int).astype(str)
    period_tag_1 = []
    for i in year_week_1:
        d = dt.datetime.strptime(i + '-1', "%Y-%W-%w")
        period_tag_1.append(d)
    df['PERIOD_TAG'] = period_tag_1
    return df



In [ ]:
dataset = make_data_continuous(df_v2)

In [ ]:
dataset_full = pd.merge(dataset,inventory_df, how = 'left', on = ['ITEM_CODE', 'CITY', 'YEAR', 'WEEK_NUM' ])
dataset_full.drop(columns = ['ITEM_CODE','Product Number'], inplace = True)

In [ ]:
dataset_full_21_23 = dataset_full[(dataset_full['YEAR'] != 2020)]
dataset_full_21_23['WEEKLY_SI_ORDER'] = dataset_full_21_23['WEEKLY_SI_ORDER'].fillna(0)
dataset_full_21_23['WEEKLY_SO_ORDER'] = dataset_full_21_23['WEEKLY_SO_ORDER'].fillna(0)
dataset_full_21_23['END_WEEK_INVENTORY'] = dataset_full_21_23['END_WEEK_INVENTORY'].fillna(0)

In [ ]:
sum(dataset_full_21_23[(dataset_full_21_23['YEAR']==2022) & (dataset_full_21_23['DP_STANDARD_SKU_DESC']=='Chilli Sauces Chinsu 1kg')]['WEEKLY_SI_ORDER'])

In [ ]:
dataset_full_21_23 = create_period_tag(dataset_full_21_23)
df_v2 = create_period_tag(df_v2)
dataset_full_21_23

In [ ]:
dataset_full_21_23['STANDARD_SKU_DESC']= dataset_full_21_23['DP_STANDARD_SKU_DESC']+'-'+dataset_full_21_23['SUB_DIVISION_DESC']+'-'+dataset_full_21_23['CATEGORY_DESC']+'-'+dataset_full_21_23['SUB_CATEGORY_DESC']


In [ ]:
df_v2['STANDARD_SKU_DESC']= df_v2['DP_STANDARD_SKU_DESC']+'-'+df_v2['SUB_DIVISION_DESC']+'-'+df_v2['CATEGORY_DESC']+'-'+df_v2['SUB_CATEGORY_DESC']


In [ ]:
last_record_week = pd.DataFrame(df_v2.groupby(['STANDARD_SKU_DESC','CITY'])['PERIOD_TAG'].agg([np.min, np.max]).reset_index())

last_record_week

In [ ]:
dataset_full_21_23 = pd.merge(dataset_full_21_23,last_record_week,how = 'left', on = ['STANDARD_SKU_DESC', 'CITY'])

In [ ]:
dataset_full_21_23=dataset_full_21_23[(dataset_full_21_23['PERIOD_TAG']>= dataset_full_21_23['amin']) 
                                     ]

In [ ]:
sum(dataset_full_21_23[(dataset_full_21_23['YEAR']==2022) & (dataset_full_21_23['DP_STANDARD_SKU_DESC']=='Chilli Sauces Chinsu 1kg')]['WEEKLY_SI_ORDER'])

In [ ]:
biggest_SKU = dataset_full_21_23.copy()
biggest_SKU = biggest_SKU.groupby(['STANDARD_SKU_DESC'])[['WEEKLY_SI_ORDER']].sum()
biggest_SKU.sort_values(by = ['WEEKLY_SI_ORDER'], ascending = False, inplace = True)
biggest_SKU.reset_index(inplace=True)

print(biggest_SKU)

In [ ]:
biggest_SKU.loc[[0,1,2,3,4,5,6,7,8,9,10,11,12],:]

In [ ]:
dataset_full_21_23.head()

In [ ]:
df2 = dataset_full_21_23.groupby(['STANDARD_SKU_DESC','PERIOD_TAG', 'CITY','YEAR', 'WEEK_NUM', 'SUB_DIVISION_DESC', 
                  'CATEGORY_DESC', 'SUB_CATEGORY_DESC']).agg({'WEEKLY_SI_ORDER' : 'sum', 'WEEKLY_SO_ORDER' : 'sum','END_WEEK_INVENTORY' : 'sum' }).reset_index()


In [ ]:
sum(df2[(df2['YEAR']==2022) & (df2['STANDARD_SKU_DESC']=='Chilli Sauces Chinsu 1kg-Seasoning-Sauces-Chilli Sauces')]['WEEKLY_SO_ORDER'])

In [ ]:
df2['PROVINCE_REGION'] = df2['CITY'] #+'-'+df2['REGION']
df2 = df2.drop(['CITY'], axis = 1)
df2['MONTH'] = df2['PERIOD_TAG'].dt.month

In [ ]:
df2 = df2.merge(Tet_Holiday, how = 'left', left_on=['WEEK_NUM', 'YEAR'], right_on= ['Iso Weeknum','Iso Year'])
df2=df2.drop(['Iso Year','Iso Weeknum'], axis= 1) #,'Holiday_count_x','Week_Year_x',	'Iso Year_y',	'Iso Weeknum_y',	'Holiday_count_y',	'Week_Year_y',	'Iso Year',	'Iso Weeknum',	'Holiday_count',	'Week_Year'
df2.rename(columns = {'Holiday_count' : 'HOLIDAY_COUNT', 'Working_day' : 'WORKING_DAY_RATE'}, inplace = True)


In [ ]:
df2 = df2.merge(Tet, how = 'left', left_on=['WEEK_NUM', 'YEAR'], right_on= ['Iso Weeknum','Iso Year'])
df2=df2.drop(['Iso Year','Iso Weeknum'], axis= 1) #,'Holiday_count_x','Week_Year_x',	'Iso Year_y',	'Iso Weeknum_y',	'Holiday_count_y',	'Week_Year_y',	'Iso Year',	'Iso Weeknum',	'Holiday_count',	'Week_Year'
df2['IS_TET'] = df2['IS_TET'].fillna(0)

In [ ]:
Moon_week = pd.read_excel('Master_Date.xlsx', sheet_name= 'Moon_week')
df2 = df2.merge(Moon_week, how = 'left', on=['WEEK_NUM', 'YEAR'])

is_1st_day_moon_cal = pd.read_excel('Master_Date.xlsx', sheet_name= '1st_day_moon_cal')
df2 = df2.merge(is_1st_day_moon_cal, how = 'left', on=['WEEK_NUM', 'YEAR'])

is_15th_day_moon_cal = pd.read_excel('Master_Date.xlsx', sheet_name= '15th_day_moon_cal')
df2 = df2.merge(is_15th_day_moon_cal, how = 'left', on=['WEEK_NUM', 'YEAR'])



In [ ]:
df2['IS_1ST_DAY_MOON_CAL'] = df2['IS_1ST_DAY_MOON_CAL'].fillna(0)
df2['IS_15TH_DAY_MOON_CAL'] = df2['IS_15TH_DAY_MOON_CAL'].fillna(0)

In [ ]:
###############################
#seasonality + date variables


def is_week(today, week_num):
    this_year, this_month = today.year, today.month
    weeks = calendar.monthcalendar(this_year, this_month)
    return 1 if today.day in weeks[week_num-1] else 0


#holiday check
def get_holiday_weeks(start_date, end_date, country):
    holidays_lis = holidays.CountryHoliday(country)
    holiday_dates = [date for date in pd.date_range(start_date, end_date) if date in holidays_lis]
    return list(set(str(date.isocalendar().week)+'-'+str(date.isocalendar().year) for date in holiday_dates))
    
# Calculate week flag
df2['IS_2ND_WEEK'] = df2['PERIOD_TAG'].apply(lambda d: is_week(d, 2))
df2['IS_3RD_WEEK'] = df2['PERIOD_TAG'].apply(lambda d: is_week(d, 3))
df2['IS_4TH_WEEK'] = df2['PERIOD_TAG'].apply(lambda d: is_week(d, 4))

# Calculate holiday flags
from isoweek import Week
def holiday_check(l_date, u_date, country, df):
    WEEK_NUM_YEAR = week_by_year(df['YEAR'].unique())
    year_week = WEEK_NUM_YEAR['YEAR'].astype(int).astype(str) +'-' + WEEK_NUM_YEAR['WEEK_NUM'].astype(int).astype(str)
    period_tag = []

    for i, row in WEEK_NUM_YEAR.iterrows():
        d = Week(int(row['YEAR']), int(row['WEEK_NUM'])).thursday()  #dt.datetime.strptime(i + '-1', "%Y-%W-%w")
        period_tag.append(d)

    WEEK_NUM_YEAR['PERIOD_TAG'] = period_tag

    holiday_weeks = get_holiday_weeks(l_date, u_date, country)

    WEEK_NUM_YEAR['IS_HOLIDAY_WEEK'] = WEEK_NUM_YEAR['PERIOD_TAG'].apply(lambda d: int(str(d.isocalendar().week)+'-'+str(d.isocalendar().year) in holiday_weeks))
    # WEEK_NUM_YEAR['IS_13W_BF_HOLIDAY'] = WEEK_NUM_YEAR['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=13)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=13)).isocalendar().year) in holiday_weeks))
    # WEEK_NUM_YEAR['IS_12W_BF_HOLIDAY'] = WEEK_NUM_YEAR['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=12)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=12)).isocalendar().year) in holiday_weeks))
    # WEEK_NUM_YEAR['IS_11W_BF_HOLIDAY'] = WEEK_NUM_YEAR['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=11)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=11)).isocalendar().year) in holiday_weeks))
    WEEK_NUM_YEAR['IS_10W_BF_HOLIDAY'] = WEEK_NUM_YEAR['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=10)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=10)).isocalendar().year) in holiday_weeks))
    WEEK_NUM_YEAR['IS_9W_BF_HOLIDAY'] = WEEK_NUM_YEAR['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=9)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=9)).isocalendar().year) in holiday_weeks))
    WEEK_NUM_YEAR['IS_8W_BF_HOLIDAY'] = WEEK_NUM_YEAR['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=8)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=8)).isocalendar().year) in holiday_weeks))
    WEEK_NUM_YEAR['IS_7W_BF_HOLIDAY'] = WEEK_NUM_YEAR['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=7)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=7)).isocalendar().year) in holiday_weeks))
    WEEK_NUM_YEAR['IS_6W_BF_HOLIDAY'] = WEEK_NUM_YEAR['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=6)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=6)).isocalendar().year) in holiday_weeks))
    WEEK_NUM_YEAR['IS_5W_BF_HOLIDAY'] = WEEK_NUM_YEAR['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=5)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=5)).isocalendar().year) in holiday_weeks))
    WEEK_NUM_YEAR['IS_4W_BF_HOLIDAY'] = WEEK_NUM_YEAR['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=4)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=4)).isocalendar().year) in holiday_weeks))
    WEEK_NUM_YEAR['IS_3W_BF_HOLIDAY'] = WEEK_NUM_YEAR['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=3)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=3)).isocalendar().year) in holiday_weeks))
    WEEK_NUM_YEAR['IS_2W_BF_HOLIDAY'] = WEEK_NUM_YEAR['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=2)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=2)).isocalendar().year) in holiday_weeks))
    WEEK_NUM_YEAR['IS_1W_BF_HOLIDAY'] = WEEK_NUM_YEAR['PERIOD_TAG'].apply(lambda d: int(str((d + dt.timedelta(weeks=1)).isocalendar().week)+'-'+str((d + dt.timedelta(weeks=1)).isocalendar().year) in holiday_weeks))
    
    return WEEK_NUM_YEAR.drop(['PERIOD_TAG'],axis = 1)

# holiday feature
holiday_check = holiday_check('2021-01-01', '2026-01-01', 'VN', df2)

df2 = df2.merge(holiday_check, how = 'left', on = ['YEAR', 'WEEK_NUM'])



In [ ]:
df2['WEEK_SIN'] = np.sin((2*np.pi/52)*df2['WEEK_NUM'])
df2['WEEK_COS'] = np.cos((2*np.pi/52)*df2['WEEK_NUM'])

In [ ]:
df2['SUB_DIVISION_DESC'] = pd.factorize(df2['SUB_DIVISION_DESC'])[0]
df2['CATEGORY_DESC'] = pd.factorize(df2['CATEGORY_DESC'])[0]
df2['SUB_CATEGORY_DESC'] = pd.factorize(df2['SUB_CATEGORY_DESC'])[0]
df2 = df2.merge(Vegan, how = 'left', on = ['YEAR', 'WEEK_NUM'])
df2['IS_BIG_VEGAN_WEEK'].fillna(0, inplace = True)
df2["QUARTER"] = df2["PERIOD_TAG"].dt.quarter

df2["QUARTER"] = df2["QUARTER"].astype('category')
df2["MONTH"] = df2["MONTH"].astype('category')
df2["WEEK_NUM"] = df2["WEEK_NUM"].astype('category')

In [ ]:
#GDP Growth Rate

# assign data of lists.  
data = {'GDP_quarter': ['2021-01-01','2021-04-01','2021-07-01','2021-10-01',
                        '2022-01-01','2022-04-01','2022-07-01','2022-10-01',
                        '2023-01-01','2023-04-01','2023-07-01','2023-10-01'
                       ], 
        'GDP_Growth': [4.48, 4.72, 6.73, -6.02,
                       5.22, 5.05, 7.83, 13.71, 
                       5.92, 2.28, 4.14, 4.14]}  
  
# Create DataFrame  
GDP_Data = pd.DataFrame(data)  
GDP_Data = GDP_Data.sort_values(['GDP_quarter'], ascending=[True])

# Create lagged features
for lag in [1,2,3,4]:  # approximate days for 3, 6, 9 and 12 months
    GDP_Data[f'gdp_lag_{lag}'] = GDP_Data['GDP_Growth'].shift(lag)

GDP_Data = GDP_Data.fillna(0)    


    
GDP_Data['perc_change_lag1'] =  GDP_Data['GDP_Growth']- (GDP_Data['gdp_lag_1'])/GDP_Data['gdp_lag_1']
GDP_Data['perc_change_lag2'] =  GDP_Data['GDP_Growth']- (GDP_Data['gdp_lag_2'])/GDP_Data['gdp_lag_2']
GDP_Data['perc_change_lag3'] =  GDP_Data['GDP_Growth']- (GDP_Data['gdp_lag_3'])/GDP_Data['gdp_lag_3']
GDP_Data['perc_change_lag4'] =  GDP_Data['GDP_Growth']- (GDP_Data['gdp_lag_4'])/GDP_Data['gdp_lag_4']

GDP_Data['perc_change_lag12'] =  (GDP_Data['gdp_lag_1']-GDP_Data['gdp_lag_2'])/GDP_Data['gdp_lag_2']
GDP_Data['perc_change_lag23'] =  (GDP_Data['gdp_lag_2']-GDP_Data['gdp_lag_3'])/GDP_Data['gdp_lag_3']
GDP_Data['perc_change_lag34'] =  (GDP_Data['gdp_lag_3']-GDP_Data['gdp_lag_4'])/GDP_Data['gdp_lag_4']



#GDP_Data[["perc_change_lag1"]] = GDP_Data[["perc_change_lag1"]].fillna(0)


cols = ["perc_change_lag1","perc_change_lag2","perc_change_lag3","perc_change_lag4",
        "perc_change_lag12","perc_change_lag23","perc_change_lag34"]

for i in range(0,len(cols)):
    GDP_Data[cols[i]]= GDP_Data[cols[i]].fillna(0)
    GDP_Data[cols[i]]= GDP_Data[cols[i]].replace([np.inf, -np.inf], 0)
    

df2['calday_quarter'] = pd.PeriodIndex(df2.PERIOD_TAG, freq='q')
GDP_Data['gdp_quarter_1'] = pd.PeriodIndex(GDP_Data.GDP_quarter, freq='q')

df2 = df2.merge(GDP_Data, left_on ='calday_quarter',right_on='gdp_quarter_1', how='left')

col_drop =["gdp_quarter_1","calday_quarter","GDP_quarter"]

df2=df2.drop(col_drop, axis=1)

df2

In [ ]:
#CPI Index Vietnam
# assign data of lists.  
CPI_data = {'CPI_month': ['2021-01-01','2021-02-01','2021-03-01','2021-04-01',
                          '2021-05-01','2021-06-01','2021-07-01','2021-08-01',
                          '2021-09-01','2021-10-01','2021-11-01','2021-12-01',
                          '2022-01-01','2022-02-01','2022-03-01','2022-04-01',
                          '2022-05-01','2022-06-01','2022-07-01','2022-08-01',
                          '2022-09-01','2022-10-01','2022-11-01','2022-12-01',
                          '2023-01-01','2023-02-01','2023-03-01','2023-04-01',
                          '2023-05-01','2023-06-01','2023-07-01','2023-08-01',
                          '2023-09-01','2023-10-01','2023-11-01','2023-12-01'], 
            'CPI_index': [ 109.12, 110.87, 109.26, 109.12, 109.16, 
                          109.02, 109.75, 110.56, 110.38, 108.97, 108.79, 108.97,
                          105.28,106.33,107.08,107.28,107.68,108.42,
                          108.85,108.86,109.29,109.44,109.87,109.95,
                          110.42,110.92,110.67,110.29,110.3,110.59,
                           111.09,112.07,112.07,112.07,112.07,112.07]}
# Create DataFrame  
CPI_Df = pd.DataFrame(CPI_data)
CPI_Df = CPI_Df.sort_values(['CPI_index'], ascending=[True])



# Create lagged features (# approximate days for 3, 6, 9 and 12 months)
for lag in [1,2,3,4]:
    CPI_Df[f'cpi_lag_{lag}'] = CPI_Df['CPI_index'].shift(lag)

CPI_Df = CPI_Df.fillna(0)


CPI_Df['cpi_perc_change_lag1'] = CPI_Df['CPI_index']- (CPI_Df['cpi_lag_1'])/CPI_Df['cpi_lag_1']
CPI_Df['cpi_perc_change_lag2'] = CPI_Df['CPI_index']- (CPI_Df['cpi_lag_2'])/CPI_Df['cpi_lag_2']
CPI_Df['cpi_perc_change_lag3'] = CPI_Df['CPI_index']- (CPI_Df['cpi_lag_3'])/CPI_Df['cpi_lag_3']
CPI_Df['cpi_perc_change_lag4'] = CPI_Df['CPI_index']- (CPI_Df['cpi_lag_4'])/CPI_Df['cpi_lag_4']

CPI_Df['cpi_perc_change_lag12'] = (CPI_Df['cpi_lag_1']-CPI_Df['cpi_lag_2'])/CPI_Df['cpi_lag_2']
CPI_Df['cpi_perc_change_lag23'] = (CPI_Df['cpi_lag_2']-CPI_Df['cpi_lag_3'])/CPI_Df['cpi_lag_3']
CPI_Df['cpi_perc_change_lag34'] = (CPI_Df['cpi_lag_3']-CPI_Df['cpi_lag_4'])/CPI_Df['cpi_lag_4']



#GDP_Data[["perc_change_lag1"]] = GDP_Data[["perc_change_lag1"]].fillna(0)


cols = ["cpi_perc_change_lag1","cpi_perc_change_lag2","cpi_perc_change_lag3","cpi_perc_change_lag4",
        "cpi_perc_change_lag12","cpi_perc_change_lag23","cpi_perc_change_lag34"]

for i in range(0,len(cols)):
    CPI_Df[cols[i]]= CPI_Df[cols[i]].fillna(0)
    CPI_Df[cols[i]]= CPI_Df[cols[i]].replace([np.inf, -np.inf], 0)


df2['data_month'] = (df2['PERIOD_TAG'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))


df2['data_month']= pd.to_datetime(df2['data_month'])
CPI_Df['CPI_month']= pd.to_datetime(CPI_Df['CPI_month'])


df2 = df2.merge(CPI_Df, left_on ='data_month',right_on='CPI_month', how='left')



col_drop =["data_month","CPI_month"]

df2=df2.drop(col_drop, axis=1)
#data_final.dtypes

### training and test

In [ ]:
# data_mdl['IS_VEGAN_WEEK'].fillna(0, inplace = True)
sellout_w = 13
sellin_w = 13
target_columns = [f'SO_{horizon}W' for horizon in range(1, sellout_w+1)]+\
                    [f'SI_{horizon}W' for horizon in range(1, sellin_w+1)]


In [ ]:

###################################
#Pipeline
# Select Date as index

class Make_Date_Transformer(BaseEstimator, TransformerMixin):
    def __init__(self, date_column):
        self.date_column = date_column    
    
    def fit(self, X, y = None):
        return self    
    
    def transform(self, X, y = None):
        X_transformed = X.copy()
        X_transformed = X_transformed.set_index(self.date_column)
        X_transformed.index = pd.to_datetime(X_transformed.index)
        return X_transformed  
    
def model_evaluation(model, Xtrain, ytrain, Xtest, ytest):
    def fit_scatter_plot(X, y, set_name):
        y_fitted_values = model.predict(X)
        xmin = y.min()
        xmax = y.max()
        plt.scatter(x = y_fitted_values, y = y, alpha=0.25)
        x_line = np.linspace(xmin, xmax, 10)
        y_line = x_line
        plt.plot(x_line, y_line, 'r--')
        plt.axhline(0, color="black", linestyle="--")
        plt.xlabel('Prediction')
        plt.ylabel('True Value')
        plt.title(f'Plot of predicted values versus true values - {set_name} set')
    
    def plot_of_residuals(X, y, set_name):
        errors = model.predict(X) - np.reshape(np.array(y), (-1))
        plt.scatter(x = y, y = errors, alpha=0.25)
        plt.axhline(0, color="r", linestyle="--")
        plt.xlabel('True Value')
        plt.ylabel('Residual')
        plt.title(f'Plot of residuals - {set_name} set')
        
    def hist_of_residuals(X, y, set_name):
        errors = model.predict(X) - np.reshape(np.array(y), (-1))
        plt.hist(errors, bins = 100)
        plt.axvline(errors.mean(), color='k', linestyle='dashed', linewidth=1)
        plt.title(f'Histogram of residuals - {set_name} set')
    
    def DPA(y_true, y_pred):
        dpa = 100 - (((np.sum(np.abs(y_pred - y_true)))/(np.sum(y_true)))*100)
        return dpa

    def BIAS(y_true, y_pred):
        bias = (((np.sum(y_pred - y_true))/(np.sum(y_true)))*100)
        return bias
    
    fig = plt.figure(figsize = (16, 10))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    
    ax = fig.add_subplot(3, 2, 1)
    fit_scatter_plot(X = Xtrain, y = ytrain, set_name = 'train')
    
    ax = fig.add_subplot(3, 2, 2)
    fit_scatter_plot(X = Xtest, y = ytest, set_name = 'test')
    
    ax = fig.add_subplot(3, 2, 3)
    plot_of_residuals(X = Xtrain, y = ytrain, set_name = 'train')
    
    ax = fig.add_subplot(3, 2, 4)
    plot_of_residuals(X = Xtest, y = ytest, set_name = 'test')    

    ax = fig.add_subplot(3, 2, 5)
    hist_of_residuals(X = Xtrain, y = ytrain, set_name = 'train')   

    ax = fig.add_subplot(3, 2, 6)
    hist_of_residuals(X = Xtest, y = ytest, set_name = 'test')
    
    plt.show()
    
    y_pred_train = model.predict(Xtrain)
    print(f'RMSE train: {sqrt(mean_squared_error(ytrain, y_pred_train))}')
    print(f'DPA  train: {DPA(ytrain, y_pred_train)}')
    print(f'BIAS train: {BIAS(ytrain, y_pred_train)}')
    print()
    y_pred_test = model.predict(Xtest)
    print(f'RMSE test:  {sqrt(mean_squared_error(ytest, y_pred_test))}')
    print(f'DPA  test: {DPA(ytest, y_pred_test)}')
    print(f'BIAS test: {BIAS(ytest, y_pred_test)}')
    
    # Feature Importance
    importance = model._final_estimator.feature_importances_
    df_feature_importance=importance.argsort()
    df_feature_importance=pd.DataFrame({
        'column':Xtrain.columns[df_feature_importance],
        'importance':importance[df_feature_importance]
    })
    df_feature_importance = df_feature_importance[
                                            df_feature_importance['importance']>=0.01].copy().reset_index(drop=True)
    plt.figure(figsize=(16, 4))
    plt.barh(df_feature_importance['column'][-10:], df_feature_importance['importance'][-10:])
    plt.tick_params(axis='both', labelsize=10)
    plt.title('Model Feature Importance', size=20)
    plt.xlabel(' ', size=15)
    plt.tight_layout()
    plt.show()


In [ ]:
#transform training and testset

def create_sale_lag(df):
    for lag in range(1, 13):
        df[f'SO_LAG_{lag}'] = df['WEEKLY_SO_ORDER'].shift(lag)
    return df

def create_growth_lag(df):
    for lag in range(1, 13):
        df[f'SO_GROWTH_{lag}'] = df['SO_GROWTH'].shift(lag)
        df[f'SO_GROWTH_{lag}'].replace([np.inf,-np.inf],100,inplace=True)
    return df

def create_sale_expanding(df):
    for lag in range(1, 14):
        df[f'SO_EXPANDING_MEAN_{lag}'] = df['WEEKLY_SO_ORDER'].expanding(lag).mean()
        df[f'SO_EXPANDING_STD_{lag}'] = df['WEEKLY_SO_ORDER'].expanding(lag).std()
        df[f'SO_EXPANDING_MAX_{lag}'] = df['WEEKLY_SO_ORDER'].expanding(lag).max()
        df[f'SO_EXPANDING_MIN_{lag}'] = df['WEEKLY_SO_ORDER'].expanding(lag).min()
        
def create_sale_rolling(df):
    for lag in range(1, 14):
        df[f'SO_WINDOW_MEAN_{lag}'] = data_set['WEEKLY_SO_ORDER'].rolling(window=lag).mean()
        if lag >= 2:
            df[f'SO_WINDOW_STD_{lag}'] = df['WEEKLY_SO_ORDER'].rolling(window=lag).std()
            df[f'SO_WINDOW_MAX_{lag}'] = df['WEEKLY_SO_ORDER'].rolling(window=lag).max()
            df[f'SO_WINDOW_MIN_{lag}'] = df['WEEKLY_SO_ORDER'].rolling(window=lag).min()
            
def create_sale_ewma(df):
    for lag in range(2, 14):
        df[f'SO_EWMA_MEAN_{lag}'] = df['WEEKLY_SO_ORDER'].ewm(span=lag).mean()
        df[f'SO_EWMA_STD_{lag}'] = df['WEEKLY_SO_ORDER'].ewm(span=lag).std()  

def create_seasonality_index(df,base_year):
    avg = df[df['YEAR'] == base_year]['WEEKLY_SO_ORDER'].mean()
    df['SEASONALITY_INDEX'] = df['WEEKLY_SO_ORDER']/avg
    for lag in range(1,13):
        df[f'SEASONALITY_INDEX_LAG_{lag}'] = df['SEASONALITY_INDEX'].shift(lag)

def mdl_transform(SKU_name,province,df):
    data_set = df[(df['STANDARD_SKU_DESC'] == SKU_name) & (df['PROVINCE_REGION'] == province)] 
    
    data_set = data_set.set_index(data_set['PERIOD_TAG']).dropna()
    data_set.index = pd.to_datetime(data_set.index)

    if (SKU_name == 'Instant Noodle Kokomi Tôm 30gói 65gr-Convenience Foods-Noodles-Instant Noodle') & (province == 'T. Bắc Giang') | (j == 'T. Hải Dương'):
        data_set['WEEKLY_SO_ORDER'] = data_set['WEEKLY_SO_ORDER'].replace(0.0,np.nan).fillna(data_set['WEEKLY_SI_ORDER'])
    
    #growth feature
    data_set['SO_LAG_1'] = data_set['WEEKLY_SO_ORDER'].shift(1)
    data_set['SO_LAG_1'].fillna(0.001,inplace=True)

    data_set['SO_GROWTH'] = (data_set['WEEKLY_SO_ORDER']/data_set['SO_LAG_1'] - 1)*100
    data_set['SO_GROWTH'].replace([np.inf,-np.inf],100,inplace=True)
    data_set['SO_GROWTH'].fillna(0, inplace = True)
    
    data_set = create_growth_lag(data_set)
    data_set = create_sale_lag(data_set)
    data_set = create_sale_expanding(data_set)
    data_set = create_sale_rolling(data_set)
    data_set = create_sale_ewma(data_set)
    data_set = create_seasonality_index(data_set, 2022)
    
    #data_set['SI_CUMSUM_BY_WEEK'] = data_set.groupby(['YEAR','MONTH'])['WEEKLY_SI_ORDER'].cumsum()
#             data_set['SO_CUMSUM_BY_WEEK'] = data_set.groupby(['YEAR', 'MONTH'])['WEEKLY_SO_ORDER'].cumsum()
#             data_set['SO_CUMSUM_BY_QUARTER'] = data_set.groupby(['YEAR','QUARTER'])['WEEKLY_SO_ORDER'].cumsum()

    data_set['SO_MEAN_BY_MONTH'] = data_set.groupby(['MONTH'])['WEEKLY_SO_ORDER'].transform(lambda x: x.sum()/data_set['YEAR'].nunique())

#             data_set['INVENTORY_BEGIN_WEEK'] = data_set['END_WEEK_INVENTORY'].shift(1)
#             data_set['STOCK_AVILABILITY'] = data_set['WEEKLY_SI_ORDER'] + data_set['INVENTORY_BEGIN_WEEK']

    for col in data_set._get_numeric_data().columns.values:
        data_set[col].fillna(data_set[col].mean(), inplace=True)
    
    data_set.replace([np.inf,-np.inf],100,inplace=True)
#             data_set['SO_DIFF'] = data_set['WEEKLY_SO_ORDER'] - data_set['SO_LAG_1']

    data_set = data_set.drop(['SUB_DIVISION_DESC',	'CATEGORY_DESC',	'SUB_CATEGORY_DESC'], axis= 1)
    return data_set



In [ ]:
import lightgbm as lgb
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings("ignore")

SO_test_result = pd.DataFrame()
SO_Pred = pd.DataFrame()
SO_test_result_ETS = pd.DataFrame()
final_features_top = []
count = 0

first_test_date = dt.datetime.strptime('2023-06-30', '%Y-%m-%d')
end_test_date = dt.datetime.strptime('2023-09-30', '%Y-%m-%d')

for i in biggest_SKU.iloc[20:30,0]: #4,8 #0,1,2,3,4,5,6,
    for j in df2[(data_mdl['STANDARD_SKU_DESC'] == i)]['PROVINCE_REGION'].unique(): #
        print(f'STANDARD_SKU_DESC = {i} and PROVINCE_REGION = {j}')
        if (
            (sum(data_set[data_set['YEAR']==2023]['WEEKLY_SO_ORDER']) > 0) & \
            (sum(data_set[data_set['YEAR']==2023]['WEEKLY_SI_ORDER']) > 0) & \
            (data_set[(data_set['YEAR']>=2022)&(data_set['PERIOD_TAG'] < first_test_date)].shape[0] >= 52)):

            data_set = mdl_transform(i,j,df2)

            for horizon_prediction in target_columns[0:13]:
                start_time = time.time()

                print(f'|---------------------------\t{horizon_prediction}\t---------------------------|')
                if horizon_prediction[:2] == 'SO':
                    nr_horizon = int(horizon_prediction[3:-1])
                    type_prediction = 'Sellout'
                    target_name = 'Sellout'

                    #SO training
                print('Creating a SellOut training set')
                all_set = data_set[(data_set['YEAR'] >= 2022)].copy()
                all_set[f'WEEK_NUM_{nr_horizon}W'] =  all_set['WEEK_NUM'].shift(-nr_horizon)
#                     all_set[f'MOON_WEEK_{nr_horizon}W'] =  all_set['MOON_WEEK'].shift(-nr_horizon)
#                     all_set[f'IS_1ST_DAY_MOON_CAL_{nr_horizon}W'] =  all_set['IS_1ST_DAY_MOON_CAL'].shift(-nr_horizon)
#                     all_set[f'IS_15TH_DAY_MOON_CAL_{nr_horizon}W'] =  all_set['IS_15TH_DAY_MOON_CAL'].shift(-nr_horizon)

                all_set[f'WEEK_SIN_{nr_horizon}W'] =  all_set['WEEK_SIN'].shift(-nr_horizon)
                all_set[f'WEEK_COS_{nr_horizon}W'] =  all_set['WEEK_COS'].shift(-nr_horizon)
                all_set[f'HOLIDAY_COUNT_{nr_horizon}W'] =  all_set['HOLIDAY_COUNT'].shift(-nr_horizon)
                all_set[f'IS_HOLIDAY_WEEK_{nr_horizon}W'] =  all_set['IS_HOLIDAY_WEEK'].shift(-nr_horizon)
                all_set[f'IS_4W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_4W_BF_HOLIDAY'].shift(-nr_horizon)
                all_set[f'IS_3W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_3W_BF_HOLIDAY'].shift(-nr_horizon)
                all_set[f'IS_2W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_2W_BF_HOLIDAY'].shift(-nr_horizon)
                all_set[f'IS_1W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_1W_BF_HOLIDAY'].shift(-nr_horizon)
                all_set[f'IS_5W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_5W_BF_HOLIDAY'].shift(-nr_horizon)
                all_set[f'IS_6W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_6W_BF_HOLIDAY'].shift(-nr_horizon)
                all_set[f'IS_7W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_7W_BF_HOLIDAY'].shift(-nr_horizon)
                all_set[f'IS_8W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_8W_BF_HOLIDAY'].shift(-nr_horizon)
                all_set[f'IS_9W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_9W_BF_HOLIDAY'].shift(-nr_horizon)
                all_set[f'IS_10W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_10W_BF_HOLIDAY'].shift(-nr_horizon)
#                     all_set[f'IS_11W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_11W_BF_HOLIDAY'].shift(-nr_horizon)
#                     all_set[f'IS_12W_BF_HOLIDAY_{nr_horizon}W'] =  all_set['IS_12W_BF_HOLIDAY'].shift(-nr_horizon)

                all_set[horizon_prediction] = all_set['WEEKLY_SO_ORDER'].shift(-nr_horizon).copy()

                all_set = all_set.dropna()

                if nr_horizon <=5 :

                    SO_train_set = all_set.loc[(all_set['PERIOD_TAG'] < first_test_date), :]
                    X_train = SO_train_set.drop(list(all_set.filter(regex='SI_'))+[horizon_prediction]+['END_WEEK_INVENTORY','PERIOD_TAG', 'STANDARD_SKU_DESC', 'PROVINCE_REGION','YEAR'], axis = 1).copy()
                    y_train = SO_train_set[horizon_prediction].copy()
                    print(X_train.columns)

                    print('--------------------------------------------------------')

                    ##### Sell Out Test ##list(all_set.filter(regex='SI_'))+

                    SO_test_set = all_set.loc[(all_set['PERIOD_TAG'] >= first_test_date)  & \
                                                (all_set['PERIOD_TAG'] <= end_test_date ), :]
                    X_test =  SO_test_set.drop(list(all_set.filter(regex='SI_'))+[horizon_prediction]+['END_WEEK_INVENTORY','PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION','YEAR'], axis = 1).copy()
                    y_test =  SO_test_set[horizon_prediction].copy()
                else:
                    historical_vars_window_5 = list(all_set.filter(regex='_1'))+list(all_set.filter(regex='_2'))+list(all_set.filter(regex='_3'))+list(all_set.filter(regex='_4'))+list(all_set.filter(regex='_5'))

                    SO_train_set = all_set.loc[(all_set['PERIOD_TAG'] < first_test_date), :]
                    X_train = SO_train_set.drop(list(all_set.filter(regex='SI_'))+ historical_vars_window_5 + [horizon_prediction]+['END_WEEK_INVENTORY','PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION','YEAR'], axis = 1).copy()
                    y_train = SO_train_set[horizon_prediction].copy()
                    print(X_train.columns)

                    print('--------------------------------------------------------')

                    ##### Sell Out Test ##

                    SO_test_set = all_set.loc[(all_set['PERIOD_TAG'] >= first_test_date)  & \
                                                (all_set['PERIOD_TAG'] <= end_test_date ), :]
                    X_test =  SO_test_set.drop(list(all_set.filter(regex='SI_')) + historical_vars_window_5 + [horizon_prediction]+['END_WEEK_INVENTORY', 'PERIOD_TAG','STANDARD_SKU_DESC', 'PROVINCE_REGION','YEAR'], axis = 1).copy()
                    y_test =  SO_test_set[horizon_prediction].copy()


                ################################################################ Add a forecast to a dataset ####################
                ######## LightGBM ##########
                param_grid = {
                    'task': ['train'], 
                    'boosting_type': ['gbdt'],
                    'objective': ['regression_l1'],
                    'metric': ['rmse'],
                    'n_estimators': [100],
                    'max_depth': [-1],
                    'feature_fraction': [0.1,0.9],
                    'bagging_fraction': [0.7],
                    'num_leaves': [10,50],
                    'learning_rate': [0.05],
                    'reg_alpha': [0.01, 0.1, 0.5]
                }

                param_candidates = ParameterGrid(param_grid)
                print(f'{len(param_candidates)} candidates')

                estimator = lgb.LGBMRegressor(metric='mape', 
                                              n_jobs=-1, 
                                              verbose=-1,
                                              random_state=123, 
                                              num_boost_round = 300,
                                              early_stopping_rounds = 20)
                def fit_model(params):
                    model1 = estimator.set_params(**params)

                    model1.fit(X_train, y_train,eval_set=[(X_train, y_train)])
                    score = model1.score(X_test, y_test)
                    return [params, score]

                results = Parallel(n_jobs = -1, backend = "threading")(delayed(fit_model)(params) for params in param_candidates)
                print(max(results, key=lambda x: x[1]))

                final_params = max(results, key=lambda x: x[1])[0]
                R2 = max(results, key=lambda x: x[1])[1]


                model = estimator.set_params(**final_params)

                model.fit(X_train, y_train,eval_set=[(X_train, y_train)])

                f_i = list(zip(X_train.columns.values,model.feature_importances_))
                f_i.sort(key = lambda x : x[1])

                final_features = [i[0] for i in f_i][-20:]

                final_features_top = final_features_top + final_features

        #         y_pred_train = model.predict(X_train)

                SO_test_set[f'Pred_{horizon_prediction}'] =  model.predict(X_test)

                SO_Pred[f'Pred_{horizon_prediction}'] = SO_test_set[f'Pred_{horizon_prediction}'].copy()

                SO_Pred[f'{horizon_prediction}'] =  SO_test_set[horizon_prediction].copy()


                print(f"Forecast Accuracy RF MAPE: {mean_absolute_percentage_error(y_test, SO_test_set[f'Pred_RF_{horizon_prediction}'])}")


                plt.figure(figsize=(25,6))
                plt.plot(range(0,len(y_test)),y_test, color = 'red', linewidth=2.0, alpha = 0.6)
                plt.plot(range(0,len(y_test)), SO_test_set[f'Pred_RF_{horizon_prediction}'], color = 'blue', linewidth=0.8)
                plt.legend(['Actual SO','Predicted SO' ])
                plt.xlabel('Timestamp')
                plt.title(f"{i} - {j} - Actual Sell Out vs Predicted Sell out")
                plt.show()

            SO_test = all_set.loc[(all_set['PERIOD_TAG'] >= first_test_date)  & \
                                        (all_set['PERIOD_TAG'] <= end_test_date ), :].drop(['SO_13W'],axis = 1)

            SO_test = pd.concat([SO_test['STANDARD_SKU_DESC'],SO_Pred], axis = 1)
            SO_test_result = pd.concat([SO_test_result, SO_test])
            SO_Pred = pd.DataFrame()

        elif ((sum(data_set[data_set['YEAR']==2023]['WEEKLY_SO_ORDER']) > 0) 
            & (data_set[(data_set['YEAR']>=2022)&(data_set['PERIOD_TAG'] < first_test_date)].shape[0] < 52)
             & (data_set[(data_set['YEAR']>=2022)&(data_set['PERIOD_TAG'] < first_test_date)].shape[0] > 8)):
    # & (data_set[(data_set['YEAR']>=2022)&(data_set['PERIOD_TAG'] < first_test_date)].shape[0] > 13)                          
            ETS_first_test_date = dt.datetime.strptime('2023-08-31', '%Y-%m-%d')


            data_set = data_set.set_index(data_set['PERIOD_TAG'])
            data_set.index = pd.to_datetime(data_set.index)

            SO_train_set = data_set[data_set['PERIOD_TAG'] < ETS_first_test_date]
            SO_train = pd.Series(SO_train_set['WEEKLY_SO_ORDER'])

            SO_test_set = data_set[(data_set['PERIOD_TAG'] >= ETS_first_test_date)&(data_set['PERIOD_TAG'] <= ETS_first_test_date+dt.timedelta(weeks=13))]
            SO_test = pd.Series(SO_test_set['WEEKLY_SO_ORDER'])
            
#             index = pd.date_range("2023-03-20", "2023-07-31", freq='W-MON')
#             SO_train_1 = pd.Series(SO_train, index=index)
            # fit in statsmodels
            from statsmodels.tsa.exponential_smoothing.ets import ETSModel
            model = ETSModel(SO_train, 
                seasonal="add",
                seasonal_periods=4,
                initial_level=SO_train.mean(), 
                            )
            fit = model.fit()

            SO_pred = fit.predict(start = '2023-08-31', end = '2023-10-30')
            SO_test_set['SO_ETS_pred'] = SO_pred
            SO_test_result_ETS = pd.concat([SO_test_result_ETS,SO_test_set[['STANDARD_SKU_DESC','SO_ETS_pred','WEEKLY_SO_ORDER']]])




  



In [ ]:
for horizon_prediction in target_columns[0:13]:
    SO_test_result[f'Actual_Pred_Diff_{horizon_prediction}'] = (SO_test_result[horizon_prediction]-SO_test_result[f'Pred_RF_{horizon_prediction}']).abs()/SO_test_result[horizon_prediction]

In [ ]:
SO_test_result.to_csv('Test_v2101_20sku.csv')